# Import liberaries 

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import math
import plotly.graph_objects as go
import time
import sys

from collections import deque
from utils import *


# Load Data

In [2]:
dataset_df=pd.read_csv("Dataset.csv")

# Explore Data

In [3]:
# The whole world's real data
dataset_df.head()

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,43.449902,39.956600,89,Kazan,Russia,55.606201,49.278702,411,1506.825604,2.330543,172.662342
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,46.283298,48.006302,-65,Kazan,Russia,55.606201,49.278702,411,1040.438320,1.798405,132.955783
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,55.305801,61.503300,769,Kazan,Russia,55.606201,49.278702,411,770.508500,1.906214,78.543730
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,55.408798,37.906300,588,Kazan,Russia,55.606201,49.278702,411,715.649350,1.699983,170.633416
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,50.643799,36.590099,735,Kazan,Russia,55.606201,49.278702,411,1008.253110,1.573957,183.681933


In [4]:
dataset_df.size

584320

In [5]:
dataset_df.shape

(36520, 16)

In [6]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36520 entries, 0 to 36519
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Airline                       36520 non-null  object 
 1   SourceAirport                 36520 non-null  object 
 2   DestinationAirport            36520 non-null  object 
 3   SourceAirport_City            36520 non-null  object 
 4   SourceAirport_Country         36520 non-null  object 
 5   SourceAirport_Latitude        36520 non-null  float64
 6   SourceAirport_Longitude       36520 non-null  float64
 7   SourceAirport_Altitude        36520 non-null  int64  
 8   DestinationAirport_City       36520 non-null  object 
 9   DestinationAirport_Country    36520 non-null  object 
 10  DestinationAirport_Latitude   36520 non-null  float64
 11  DestinationAirport_Longitude  36520 non-null  float64
 12  DestinationAirport_Altitude   36520 non-null  int64  
 13  D

In [7]:
# The ranges of latitude, longitude, and altitude are naturally occurring
dataset_df.describe()

,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
count,36520.000000,36520.000000,36520.000000,36520.000000,36520.000000,36520.000000,36520.000000,36520.000000,36520.000000
mean,32.296961,8.902498,743.114595,32.297468,8.919798,746.644907,1772.588106,2.701073,227.204240
std,21.665060,77.269967,1430.097451,21.672549,77.269350,1435.930879,1943.919543,2.432245,196.240579
min,-54.843300,-179.876999,-72.000000,-54.843300,-179.876999,-72.000000,0.000000,0.035115,3.239360
25%,24.957600,-68.363403,41.000000,24.957600,-68.268501,41.000000,584.226246,1.240990,109.999919
50%,37.618999,9.221960,184.000000,37.618999,9.276740,184.000000,1168.593635,1.976535,169.628714
75%,47.121899,58.284401,681.000000,46.991100,58.284401,681.000000,2104.132617,3.147645,264.418329
max,78.246101,179.341003,14472.000000,78.246101,179.341003,14472.000000,13808.178254,17.646250,1451.239697


In [8]:
df=dataset_df.copy()

# Problem Formulation

### Problem class 

In [9]:
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

        # def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

### Node class

In [10]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0): # Default values for root node
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost #Total path cost--> g -- c in Problem class
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:                  # Back track from current node to the root
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)

## Graph

In [11]:
class Graph:
    """A graph connects nodes (vertices) by edges (links). Each edge can also
    have a length associated with it. The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C. You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more links with g.connect('B', 'C', 3), then
    inverse link is also added. You can use g.nodes() to get a list of nodes,
    g.get('A') to get a dict of links out of A, and g.get('A', 'B') to get the
    length of the link from A to B. 'Lengths' can actually be any object at
    all, and nodes can be any hashable object."""

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.connect1(A, B, distance)
        if not self.directed:
            self.connect1(B, A, distance)

    def connect1(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        """Return a link distance or a dict of {node: distance} entries.
        .get(a,b) returns the distance or None;
        .get(a) returns a dict of {node: distance} entries, possibly {}."""
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

In [12]:
# For bidirectional graph
def UndirectedGraph(graph_dict=None):
    """Build a Graph where every edge (including future ones) goes both ways."""
    return Graph(graph_dict=graph_dict, directed=False)

In [13]:
# Subclass from Problem class to Build specific problem definitions (Flight trip)
class GraphProblem(Problem):
    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph):
        super().__init__(initial, goal)
        self.graph = graph

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        return list(self.graph.get(A).keys())

    def result(self, state, action):
        """The result of going to a neighbor is just that neighbor."""
        return action

    def path_cost(self, cost_so_far, A, action, B):
        return cost_so_far + (self.graph.get(A, B) or np.inf)    # get(a,b) --> get the distance between node a and b

    def find_min_edge(self):
        """Find minimum value of edges."""
        m = np.inf
        for d in self.graph.graph_dict.values():
            local_min = min(d.values())
            m = min(m, local_min)

        return m
# Write your heuristac function (3d distance)
    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(distance(locs[node], locs[self.goal]))

            return int(distance(locs[node.state], locs[self.goal]))
        else:
            return np.inf      # Heuristic for unavailable node

In [14]:
# Our heuristac (the stright line distance)
def distance(source, destination):
    d1_source, d2_source, d3_source=source
    d1_destination, d2_destination, d3_destination=destination

    heuristic = math.sqrt(((d1_source - d1_destination) ** 2) + ((d2_source - d2_destination) ** 2) +
                          ((d3_source - d3_destination) ** 2))
    return heuristic

## Construct flight_trip problem Graph 

In [15]:
# Build node and successors dicationary for pass it to the graph

world_dict={}
for source_airport in list(df['SourceAirport'].unique()):
    source_to_destinations_df=df[df['SourceAirport']==source_airport][['SourceAirport','DestinationAirport','Distance']]

    destinations_dict={}
    for index, row in source_to_destinations_df.iterrows():
        destinations_dict.update({row['DestinationAirport']: row['Distance']})

    world_dict.update({source_airport: destinations_dict})

In [16]:
# Instantiate Undirected Graph
world_map=UndirectedGraph(world_dict)

In [17]:
# Locations in Latitude, Longitude and Altitude for calculate Heuristac function
locations_dict={}
for nod in list(df['SourceAirport'].unique()):
    nod_df=df[df['SourceAirport']==nod][['SourceAirport_Latitude','SourceAirport_Longitude','SourceAirport_Altitude']]
    locations_dict.update({nod:tuple(nod_df.iloc[0])})

In [18]:
# Handle all Airpots locations -- sources and destinations
for nod in list(df['DestinationAirport'].unique()):
    nod_df=df[df['DestinationAirport']==nod][['DestinationAirport_Latitude','DestinationAirport_Longitude','DestinationAirport_Altitude']]
    if nod not in locations_dict.keys():
        locations_dict.update({nod:tuple(nod_df.iloc[0])})

In [19]:
# Pass a new attribute locations
world_map.locations=locations_dict

In [20]:
# Pass intial and goal states  
airport_intial='Imam Khomeini International Airport'
airport_goal='Raleigh Durham International Airport'
# Instantiate Our Problem
world_problem = GraphProblem(airport_intial , airport_goal , world_map)

# Visualization

In [21]:
# Locations in Latitude and Longitude for Visualiztion
locations_2d_dict={}
for nod in list(df['SourceAirport'].unique()):
    nod_df=df[df['SourceAirport']==nod][['SourceAirport_Latitude','SourceAirport_Longitude']]
    locations_2d_dict.update({nod:tuple(nod_df.iloc[0])})

In [22]:
# Handle all Airpots locations -- sources and destinations
for nod in list(df['DestinationAirport'].unique()):
    nod_df=df[df['DestinationAirport']==nod][['DestinationAirport_Latitude','DestinationAirport_Longitude']]
    if nod not in locations_2d_dict.keys():
        locations_2d_dict.update({nod:tuple(nod_df.iloc[0])})

In [23]:
# Pass a new attribute locations_2d
world_map.locations_2d=locations_2d_dict

In [24]:
# Draw the path solution for the problem
def draw_path (lat_list,lon_list):
    fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lon_list,
    lat = lat_list,
    marker = {'size': 10}))

    fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "open-street-map",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1})

    return fig

In [25]:
# Add another path solution for your figure
def add_trace_path(fig,lat_list,lon_list,name=None):
    fig.add_trace(go.Scattermapbox(
    mode = "markers+text+lines",
    lon = lon_list,
    lat = lat_list,
    text=name,
    marker = {'size': 10}))
    return fig

# Uninformed Search

## Breadth First Search --Graph version

In [26]:
def breadth_first_graph_search(problem):
    """[Figure 3.11]
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier = deque([node])
    explored = set()
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

In [27]:
# Execute algorithm and calculate execution time
start_time = time.time()
breadth_node=breadth_first_graph_search(world_problem)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 0.13123s


In [28]:
# Sequence of actions from the initial node to the Goal
print(breadth_node.solution())

['London Heathrow Airport', 'Raleigh Durham International Airport']


In [29]:
# Visualiztion the solution path
path_states=[]
for node in breadth_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

## Depth First Search -- graph version

In [30]:
def depth_first_graph_search(problem):
    """
    [Figure 3.7]
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Does not get trapped by loops.
    If two paths reach a state, only use the first one.
    """
    frontier = [(Node(problem.initial))]  # Stack

    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
    return None

In [31]:
# Execute algorithm and calculate execution time
start_time = time.time()
depth_first_node=depth_first_graph_search(world_problem)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 0.62173s


In [32]:
# Sequence of actions from the initial node to the Goal
print(depth_first_node.solution())

['Mazar I Sharif Airport', 'Mashhad International Airport', 'King Abdulaziz International Airport', 'Mattala Rajapaksa International Airport', 'Malé International Airport', 'Cochin International Airport', 'Pune Airport', 'Netaji Subhash Chandra Bose International Airport', 'Kunming Changshui International Airport', 'Don Mueang International Airport', 'Krabi Airport', 'Samui Airport', 'Sultan Abdul Aziz Shah International Airport', 'Hang Nadim International Airport', 'Supadio Airport', 'Kuching International Airport', 'Kota Kinabalu International Airport', 'Tawau Airport', 'Juwata Airport', 'Hasanuddin International Airport', 'Sam Ratulangi Airport', 'Naha Airport', 'Sendai Airport', 'Daniel K Inouye International Airport', 'Bellingham International Airport', 'Phoenix-Mesa-Gateway Airport', 'Chicago Rockford International Airport', 'Charlotte County Airport', 'Youngstown Warren Regional Airport', 'Myrtle Beach International Airport', 'Arnold Palmer Regional Airport', 'Orlando Internatio

In [33]:
# Visualiztion the path
path_states=[]
for node in depth_first_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

## Depth Limited Search -- graph version

In [34]:
def depth_limited_search(problem, limit=50):
    """[Figure 3.17]"""

    def recursive_dls(node, problem, limit):
        if problem.goal_test(node.state):
            return node
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)

In [35]:
# Execute algorithm and calculate execution time
start_time = time.time()
depth_limited_node=depth_limited_search(world_problem)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 0.01227s


In [36]:
# Sequence of actions from the initial node to the Goal
print(depth_limited_node.solution())

['Heydar Aliyev International Airport', 'Mineralnyye Vody Airport', 'Astrakhan Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', 'Kazan International Airport', 'Belgorod International Airport', '

In [37]:
# Visualiztion the path
path_states=[]
for node in depth_limited_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

## Iterative Deepening Search

In [38]:
def iterative_deepening_search(problem):
    """[Figure 3.18]"""
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result

In [39]:
# Execute algorithm and calculate execution time
start_time = time.time()
iterative_deepening_node=iterative_deepening_search(world_problem)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 0.00573s


In [40]:
# Sequence of actions from the initial node to the Goal
print(iterative_deepening_node.solution())

['London Heathrow Airport', 'Raleigh Durham International Airport']


In [41]:
# Visualiztion the path
path_states=[]
for node in iterative_deepening_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

# Best First Search -- Graph versions

In [42]:
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)

    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()

        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)

    return None

## Uniform Cost Search

In [43]:
# f=g=distance
def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost, display)


In [44]:
# Execute algorithm and calculate execution time
start_time = time.time()
uniform_cost_node=uniform_cost_search(world_problem)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 3.41420s


In [45]:
# Sequence of actions from the initial node to the Goal
print(uniform_cost_node.solution())

['Zvartnots International Airport', 'Václav Havel Airport Prague', 'Newcastle Airport', 'Melbourne International Airport', 'Charlotte Douglas International Airport', 'Raleigh Durham International Airport']


In [46]:
# Visualiztion the path
path_states=[]
for node in uniform_cost_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

## Greedy best first search

In [47]:
#Greedy best-first search is accomplished by specifying f(n) = h(n)
greedy_best_first_graph_search = best_first_graph_search


In [48]:
# Execute algorithm and calculate execution time
start_time = time.time()
greedy_node=greedy_best_first_graph_search(world_problem,world_problem.h)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 0.18703s


In [49]:
# Sequence of actions from the initial node to the Goal
print(greedy_node.solution())

['Frankfurt am Main Airport', 'Seattle Tacoma International Airport', 'Austin Bergstrom International Airport', 'Memphis International Airport', 'Raleigh Durham International Airport']


In [50]:
# Visualiztion the path
path_states=[]
for node in greedy_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

## A* Search

In [51]:
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

In [52]:
# Execute algorithm and calculate execution time
start_time = time.time()
astar_node=astar_search(world_problem,world_problem.h)
elapsed_time = time.time()  - start_time
minutes, seconds = divmod(elapsed_time, 60)
print("Execution Time: {:.0f}m {:.5f}s".format(minutes, seconds))

Execution Time: 0m 3.62665s


In [53]:
# Sequence of actions from the initial node to the Goal
print(astar_node.solution())

['Zvartnots International Airport', 'Václav Havel Airport Prague', 'Newcastle Airport', 'Melbourne International Airport', 'Charlotte Douglas International Airport', 'Raleigh Durham International Airport']


In [54]:
# Visualiztion the path
path_states=[]
for node in astar_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)
path_fig.show()

# Searches Comparison different paths with add_trace

In [55]:
start_time = time.time()
breadth_node=breadth_first_graph_search(world_problem)
elapsed_time = time.time() - start_time

path_states=[]
for node in breadth_node.path():
    path_states.append(node.state)

lat_list=[world_map.locations_2d[state][0] for state in path_states ]
lon_list=[world_map.locations_2d[state][1] for state in path_states ]

path_fig=draw_path(lat_list,lon_list)

algorithms_elapsed_times = {'breadth first':elapsed_time}
search_algorthms={'depth first': depth_first_graph_search,
                  'depth limited': depth_limited_search,
                  'iterative deepening': iterative_deepening_search,
                   'uniform cost': uniform_cost_search,
                    'greedy': greedy_best_first_graph_search,
                    'A*': astar_search}
for name,search in search_algorthms.items():
    start_time = time.time()
    if search not in [greedy_best_first_graph_search,astar_search]:
        search_node=search(world_problem)
    else:
        search_node=search(world_problem,world_problem.h)
    elapsed_time = time.time() - start_time
    algorithms_elapsed_times.update({name: elapsed_time})
    
    path_states=[]
    for node in search_node.path():
        path_states.append(node.state)

    lat_list=[world_map.locations_2d[state][0] for state in path_states ]
    lon_list=[world_map.locations_2d[state][1] for state in path_states ]

    path_fig=add_trace_path(path_fig,lat_list,lon_list,name)

path_fig.show()

# Compare Execution time for each algorithm


In [56]:
fig_algorithms_elapsed_times = px.bar(
    x=list(algorithms_elapsed_times.keys()),
    y=list(algorithms_elapsed_times.values()),
    color=list(algorithms_elapsed_times.keys()),
    title="Algorithms Execution Elapsed Time",
    labels={"x": "Algorithm", "y": "Time in seconds"},
)
fig_algorithms_elapsed_times.show()